# 5. Data Manipulation III - Grouping and Aggregation

The goal of this module is to become familiar with grouping and aggregation. The two main topics we'll cover are:
1. `pl.DataFrame.group_by`, the main entrypoint to group operations, along with `.agg`, the way to compute aggregations within groups.
2. Using `polars`'s implementation of window functions, `pl.Expr.over()`.
3. Two way group-by's: pivot tables.

But first we import `polars`...

In [1]:
import polars as pl

... and load the data, this time using the name mapping from the last module.

In [2]:
column_rename_mapping = {
    "VendorID": "vendor_id",
    "RatecodeID": "ratecode_id",
    "PULocationID": "pu_location_id",
    "DOLocationID": "do_location_id",
    "Airport_fee": "airport_fee",
}
df = (
    pl.read_parquet("../data/yellow_tripdata_2024-03.parquet")
    .rename(column_rename_mapping)
)
df.head()

vendor_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecode_id,store_and_fwd_flag,pu_location_id,do_location_id,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
i32,datetime[ns],datetime[ns],i64,f64,i64,str,i32,i32,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,2024-03-01 00:18:51,2024-03-01 00:23:45,0,1.3,1,"""N""",142,239,1,8.6,3.5,0.5,2.7,0.0,1.0,16.3,2.5,0.0
1,2024-03-01 00:26:00,2024-03-01 00:29:06,0,1.1,1,"""N""",238,24,1,7.2,3.5,0.5,3.0,0.0,1.0,15.2,2.5,0.0
2,2024-03-01 00:09:22,2024-03-01 00:15:24,1,0.86,1,"""N""",263,75,2,7.9,1.0,0.5,0.0,0.0,1.0,10.4,0.0,0.0
2,2024-03-01 00:33:45,2024-03-01 00:39:34,1,0.82,1,"""N""",164,162,1,7.9,1.0,0.5,1.29,0.0,1.0,14.19,2.5,0.0
1,2024-03-01 00:05:43,2024-03-01 00:26:22,0,4.9,1,"""N""",263,7,2,25.4,3.5,0.5,0.0,0.0,1.0,30.4,2.5,0.0


## 5.1. Grouping Dataframes

As we've progressed through previous modules, we've seen how to do some basic aggregations on an entire dataframe, with functions such as `.sum()`, `.min()`, `.max()`, `.count()`, and `.mean()`. Collecting some of the examples from previous modules, we can begin to see a short summary about our dataset:

In [3]:
(
    df
    .select(
        pl.len().alias("count_trips"),  # pl.len() just gives the length of the whole dataframe
        pl.col("passenger_count").max().name.suffix("_max"),
        pl.col("trip_distance").min().name.suffix("_min"),
        pl.col("trip_distance").max().name.suffix("_max"),
        pl.col("trip_distance").mean().name.suffix("_mean"),
    )
)

count_trips,passenger_count_max,trip_distance_min,trip_distance_max,trip_distance_mean
u32,i64,f64,f64,f64
3582628,9,0.0,176836.3,4.517412


One taxi ride had 9 passengers? That's something we haven't seen yet, that's a lot of passengers!

But what if we wanted to know these statistics for the different pickup location IDs? To do that, we need to crack open the `group_by` functionality.

In [4]:
(
    df
    .group_by("pu_location_id")
    .agg(
        pl.len().alias("count_trips"),
        pl.col("passenger_count").max().name.suffix("_max"),
        pl.col("trip_distance").min().name.suffix("_min"),
        pl.col("trip_distance").max().name.suffix("_max"),
        pl.col("trip_distance").mean().name.suffix("_mean"),
    )
    .sort(pl.col("count_trips"), descending=True)
    .head()
)

pu_location_id,count_trips,passenger_count_max,trip_distance_min,trip_distance_max,trip_distance_mean
i32,u32,i64,f64,f64,f64
161,163269,6,0.0,51066.77,2.692728
132,157706,9,0.0,9211.95,15.76677
237,155631,6,0.0,44866.77,2.096025
236,146044,6,0.0,109619.96,3.481146
162,123805,8,0.0,57408.32,3.030781


To transition from aggregate statistics on the entire dataframe with `select` to aggregrate statistics of groups with `group_by` and `agg`, we just have to put everything from the `select` statement inside an `agg`.

And, just like the other Query Statements `select`, `filter`, and `sort`, `group_by` can also receive a list of columns as input, for example if we want to group by `pu_location_id` and `do_location_id`:

In [5]:
(
    df
    .group_by(["pu_location_id", "do_location_id"])
    .agg(
        pl.len().alias("count_trips"),
        pl.col("passenger_count").max().name.suffix("_max"),
        pl.col("trip_distance").min().name.suffix("_min"),
        pl.col("trip_distance").max().name.suffix("_max"),
        pl.col("trip_distance").mean().name.suffix("_mean"),
    )
    .sort(pl.col("count_trips"), descending=True)
    .head()
)

pu_location_id,do_location_id,count_trips,passenger_count_max,trip_distance_min,trip_distance_max,trip_distance_mean
i32,i32,u32,i64,f64,f64,f64
237,236,21477,6,0.0,13.29,1.056593
236,237,19101,6,0.0,19.8,1.03706
236,236,15932,6,0.0,33.8,0.614886
237,237,15156,6,0.0,38.51,0.625747
161,237,10601,6,0.0,8.6,1.062107


It can also receive those same multiple columns as positional arguments:

In [6]:
(
    df
    .group_by("pu_location_id", "do_location_id")
    .agg(
        pl.len().alias("count_trips"),
        pl.col("passenger_count").max().name.suffix("_max"),
        pl.col("trip_distance").min().name.suffix("_min"),
        pl.col("trip_distance").max().name.suffix("_max"),
        pl.col("trip_distance").mean().name.suffix("_mean"),
    )
    .sort(pl.col("count_trips"), descending=True)
    .head()
)

pu_location_id,do_location_id,count_trips,passenger_count_max,trip_distance_min,trip_distance_max,trip_distance_mean
i32,i32,u32,i64,f64,f64,f64
237,236,21477,6,0.0,13.29,1.056593
236,237,19101,6,0.0,19.8,1.03706
236,236,15932,6,0.0,33.8,0.614886
237,237,15156,6,0.0,38.51,0.625747
161,237,10601,6,0.0,8.6,1.062107


And it can also receive computed `pl.Expr` objects as input, for example if we want to group by whether or not rides had the same pickup and dropoff zone:

In [7]:
(
    df
    .group_by(
        pl.col("pu_location_id").eq(pl.col("do_location_id")).alias("same_pu_do_zone")
    )
    .agg(
        pl.len().alias("count_trips"),
        pl.col("passenger_count").max().name.suffix("_max"),
        pl.col("trip_distance").min().name.suffix("_min"),
        pl.col("trip_distance").max().name.suffix("_max"),
        pl.col("trip_distance").mean().name.suffix("_mean"),
    )
    .sort(pl.col("count_trips"), descending=True)
    .head()
)

same_pu_do_zone,count_trips,passenger_count_max,trip_distance_min,trip_distance_max,trip_distance_mean
bool,u32,i64,f64,f64,f64
false,3401337,9,0.0,176836.3,4.705378
true,181291,9,0.0,47635.92,0.990839


Well, that makes sense--rides that started and ended in the same zone had a shorter `mean("trip distance")` than rides that started and ended in different zones.

Finally, if you don't want to do any aggregation on your groups, but instead want to use each group's dataframe for some other operation, then `.group_by()` conveniently returns a `python` `Dict` object mapping the group to that group's dataframe: 

In [8]:
df_groups_mapping = df.group_by(pl.col("pu_location_id").eq(pl.col("do_location_id")).alias("same_pu_do_zone"))

In [9]:
type(df_groups_mapping)

polars.dataframe.group_by.GroupBy

In [10]:
for group_id, group_df in df_groups_mapping:
    print(group_id)
    print(group_df.head(2))

(False,)
shape: (2, 19)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ vendor_id ┆ tpep_pick ┆ tpep_drop ┆ passenger ┆ … ┆ improveme ┆ total_amo ┆ congestio ┆ airport_ │
│ ---       ┆ up_dateti ┆ off_datet ┆ _count    ┆   ┆ nt_surcha ┆ unt       ┆ n_surchar ┆ fee      │
│ i32       ┆ me        ┆ ime       ┆ ---       ┆   ┆ rge       ┆ ---       ┆ ge        ┆ ---      │
│           ┆ ---       ┆ ---       ┆ i64       ┆   ┆ ---       ┆ f64       ┆ ---       ┆ f64      │
│           ┆ datetime[ ┆ datetime[ ┆           ┆   ┆ f64       ┆           ┆ f64       ┆          │
│           ┆ ns]       ┆ ns]       ┆           ┆   ┆           ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 1         ┆ 2024-03-0 ┆ 2024-03-0 ┆ 0         ┆ … ┆ 1.0       ┆ 16.3      ┆ 2.5       ┆ 0.0      │
│           ┆ 1         ┆ 1         ┆           ┆   ┆           ┆  

Now, we have everything we need to perform aggregrate operations within groups... but what if we want to do analytics within groups, without aggregrating? For that, we need window functions.

## 5.2. Window Functions in `polars`

`group_by` helps us to answer questions like:
- What is the maximum trip distance for each pickup zone?
- What was the average tip paid for rides that went to the airport versus didn't go to the airport?

But there is another class of questions that involve understanding within-group properties, without aggregation. For example:
- Show me the five longest trips for trips that had a tip vs didn't.
- Show me the three most expensive trips for each different payment type.

To answer such questions, SQL offers Window Functions; in `polars`, this functionality comes to us with `pl.Expr.over()`. With `pl.Expr.over()`, we create some sort of aggregration on a `pl.Expr` object, and then specify that it should be executed `over` a particular group.

Let's start with the first example, showing **the five longest trips for trips that had a tip vs didn't**, sequentially constructing the query:

We start with adding a boolean `"had_tip"` column, sorting the dataframe to more easily see the results of our query, and selecting only a few columns that we're interested in:

In [11]:
(
    df
    .with_columns(
        pl.col("tip_amount").gt(0).alias("had_tip")
    )
    .sort(pl.col("trip_distance"), descending=True)
    .select([
        "trip_distance",
        "tip_amount",
        "had_tip"
    ])
    .head()
)

trip_distance,tip_amount,had_tip
f64,f64,bool
176836.3,5.46,true
176744.79,19.28,true
176329.23,0.0,false
138097.21,14.14,true
136660.1,0.0,false


And now we add a column for `.rank()`, which assigns a rank of `1` to `n` to each row in the column (where `n` is the length of the column):

In [12]:
(
    df
    .with_columns(
        pl.col("tip_amount").gt(0).alias("had_tip")
    )
    .sort(pl.col("trip_distance"), descending=True)
    .select([
        "trip_distance",
        "tip_amount",
        "had_tip",
        pl.col("trip_distance").rank(descending=True).name.suffix("_rank")
    ])
    .head()
)

trip_distance,tip_amount,had_tip,trip_distance_rank
f64,f64,bool,f64
176836.3,5.46,true,1.0
176744.79,19.28,true,2.0
176329.23,0.0,false,3.0
138097.21,14.14,true,4.0
136660.1,0.0,false,5.0


That's great! We now have a column `"trip_distance_rank"` which reflects the rank of `"trip_distance"` from highest to lowest. But that's not exactly what we want--we want it **within** the `"had_tip"` group! It's time to use `pl.Expr.over()`.

In [13]:
(
    df
    .with_columns(
        pl.col("tip_amount").gt(0).alias("had_tip")
    )
    .sort(pl.col("trip_distance"), descending=True)
    .select([
        "trip_distance",
        "tip_amount",
        "had_tip",
        (
            pl.col("trip_distance")
            .rank(descending=True)
            .over("had_tip")
            .name.suffix("_rank_within_had_tip")
        )
    ])
    .head()
)

trip_distance,tip_amount,had_tip,trip_distance_rank_within_had_tip
f64,f64,bool,f64
176836.3,5.46,true,1.0
176744.79,19.28,true,2.0
176329.23,0.0,false,1.0
138097.21,14.14,true,3.0
136660.1,0.0,false,2.0


It's as simple as that--we just pass the column `"had_tip"` to `pl.Expr.over()`, and `polars` takes care of the rest! Finally, we want to see only the top 3 rides within the group, so we add the new column to a `.filter()` statement...

In [14]:
(
    df
    .with_columns(
        pl.col("tip_amount").gt(0).alias("had_tip")
    )
    .sort(pl.col("trip_distance"), descending=True)
    .select([
        "trip_distance",
        "tip_amount",
        "had_tip",
        (
            pl.col("trip_distance")
            .rank(descending=True)
            .over("had_tip")
            .name.suffix("_rank_within_had_tip")
        )
    ])
    .filter(
        pl.col("trip_distance_rank_within_had_tip").le(3)  # `le()` means Less than or Equal to
    )
)

trip_distance,tip_amount,had_tip,trip_distance_rank_within_had_tip
f64,f64,bool,f64
176836.3,5.46,true,1.0
176744.79,19.28,true,2.0
176329.23,0.0,false,1.0
138097.21,14.14,true,3.0
136660.1,0.0,false,2.0
109619.96,0.0,false,3.0


Voila! And, you might be getting annoyed at this point, but I have to say it here too--`pl.Expr.over()` can accept a list of columns, and it can also accept a `pl.Expr` object. So, we don't necessarily need to create the `"had_tip"` column before--we can just create it inside the call to `.over()` if we want:

In [15]:
(
    df
#     .with_columns(
#         pl.col("tip_amount").gt(0).alias("had_tip")
#     )
    .sort(pl.col("trip_distance"), descending=True)
    .select([
        "trip_distance",
        "tip_amount",
#         "had_tip",
        (
            pl.col("trip_distance")
            .rank(descending=True)
            .over(pl.col("tip_amount").gt(0))
            .name.suffix("_rank_within_had_tip")
        )
    ])
    .filter(
        pl.col("trip_distance_rank_within_had_tip").le(3)  # `le()` means Less than or Equal to
    )
)

trip_distance,tip_amount,trip_distance_rank_within_had_tip
f64,f64,f64
176836.3,5.46,1.0
176744.79,19.28,2.0
176329.23,0.0,1.0
138097.21,14.14,3.0
136660.1,0.0,2.0
109619.96,0.0,3.0


The result is the same as before.

## 5.3. Pivot Tables

Pivot tables are a critical ingredient in most data manipulation tools, such as SQL and Pandas, and the same is true for `polars`! If you're not familiar, prepare to be amazed--pivot tables take aggregrations literally to the next dimension: 
1. Simple aggregation functions inside a `.select()` call gives aggregations over the whole dataframe;
2. Aggregating with `.group_by(...).agg(...)` gives aggregations over one column as an axis;
3. Aggregating with `.pivot()` gives aggregations across two columns as two axes.

Let's learn by example, trying to understand the average tip-amount, **pivoting** by `"passenger_count"` and whether or not a **toll** was paid:

We start by creating a `"had_toll"` column:

In [19]:
(
    df
    .with_columns(pl.col("tolls_amount").gt(0).alias("had_toll"))
    .select(["passenger_count", "tolls_amount", "had_toll", "tip_amount"])
    .head()
)

passenger_count,tolls_amount,had_toll,tip_amount
i64,f64,bool,f64
0,0.0,false,2.7
0,0.0,false,3.0
1,0.0,false,0.0
1,0.0,false,1.29
0,0.0,false,0.0


And then we can plug it into `pl.DataFrame.pivot()`, setting the `aggregate_function` as `"mean"`:

In [20]:
(
    df
    .with_columns(pl.col("tolls_amount").gt(0).alias("had_toll"))
    .pivot(
        index="had_toll",
        on="passenger_count",
        values="tip_amount",
        aggregate_function="mean"
    )
    .head()
)

had_toll,0,1,4,2,3,5,6,9,8,7,null
bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
false,2.681551,2.937724,2.851806,3.008058,2.907034,3.01128,2.94938,9.95,11.28,5.98,0.488163
true,9.91655,10.384319,9.95942,10.730344,10.525271,10.742104,10.852298,21.28,24.2,0.0,6.333642


It's a bit annoying here that the columns are out of numeric order... but `polars` offers a `sort_columns` argument to handle it:

In [21]:
(
    df
    .with_columns(pl.col("tolls_amount").gt(0).alias("had_toll"))
    .pivot(
        index="had_toll",
        on="passenger_count",
        values="tip_amount",
        aggregate_function="mean",
        sort_columns=True
    )
    .head()
)

had_toll,0,1,2,3,4,5,6,7,8,9,null
bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
false,2.681551,2.937724,3.008058,2.907034,2.851806,3.01128,2.94938,5.98,11.28,9.95,0.488163
true,9.91655,10.384319,10.730344,10.525271,9.95942,10.742104,10.852298,0.0,24.2,21.28,6.333642


That's easier to read! And now we can focus on the result, and see that there's a clear upward trend--trips with tolls had higher tips, across all values for `"passenger_count"`. And it seems like there might be an upward trend in `mean("tip_amount")` as a function of `"passenger_count"`... but somehow the data in the high values of `"passenger_count"` looks noisy. To be more confident, we might want to check the amount of data for those high values of `"passenger_count"`, which we can do by changing the `aggregate_function` from `"mean"` to `"len"`:

In [22]:
(
    df
    .with_columns(pl.col("tolls_amount").gt(0).alias("had_toll"))
    .pivot(
        index="had_toll",
        on="passenger_count",
        values="tip_amount",
        aggregate_function="len",
        sort_columns=True
    )
    .head()
)

had_toll,0,1,2,3,4,5,6,7,8,9,null
bool,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
false,38285,2257770,404925,96786,57555,30248,20758,5,9,2,416092
true,2087,176671,48972,11217,7344,2452,1349,1,1,1,10098


And we guessed right! Very few rides had such high `"passenger_count"`s; it's hard to take seriously averages from so few data points.

As a final note, `.pivot()` shares the same behavior as `.over()`, `.select()`, `.filter()`, `.sort()`, etc--it can accept lists of columns as inputs for `index` and `columns`. For example, if we want to pivot by `"passenger_count"` with the combination of `"pu_location_id"` and `"do_location_id"`:

In [23]:
(
    df
    .with_columns(pl.col("tolls_amount").gt(0).alias("had_toll"))
    .pivot(
        index=["pu_location_id", "do_location_id"],
        on="passenger_count",
        values="tip_amount",
        aggregate_function="len",
        sort_columns=True
    )
    .head()
)

pu_location_id,do_location_id,0,1,2,3,4,5,6,7,8,9,null
i32,i32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
142,239,144,7148,1419,317,175,98,80,null,null,null,402
238,24,18,1052,138,35,11,15,9,null,null,null,94
263,75,39,2652,406,83,33,42,28,null,null,null,374
164,162,32,1828,311,80,56,19,14,null,null,null,177
263,7,3,125,12,3,1,1,1,null,null,null,5


## Conclusion

In this module, we've learned everything we need to in order to get up and running with grouping and aggregating data.